In [ ]:
import tensorflow as tf
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
import joblib
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import SequentialFeatureSelector
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from kerastuner.tuners import RandomSearch
#from imblearn.over_sampling import SMOTE

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('./merged_data.csv')
df

In [ ]:
target = df['type']
target

In [ ]:
target.replace({0: 1, 1: 0, 2: 0,3:0},inplace=True)
target

In [ ]:
#replace 1 with "Fire" and 0 with "No Fire" 
target.replace({1: "Fire", 0: "No Fire"},inplace=True)
target

In [ ]:
#add gap between bars and add number on top of bars
sns_hist = sns.histplot(target, stat='count', discrete=True)
plt.title('Distribution of Target Variable') 
plt.xlabel('Type')

colors = ['red', 'blue']
for i, bar in enumerate(sns_hist.patches):
    bar.set_color(colors[i])
    bar.set_edgecolor('black') 
    bar.set_linewidth(1)

sns_hist.bar_label(sns_hist.containers[0])
plt.show()

In [ ]:
if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
!nvidia-smi

# ***Data Preprocessing***

In [ ]:
# Load the dataset
data = pd.read_csv('./merged_data.csv')#
data

In [ ]:
timestamps = data['acq_time']

def getCosSinHour(df):

    date = pd.to_datetime(data['acq_date'] +'T'+ data['acq_time'])

    hour = date.dt.hour

    cos_hour = np.cos(2 * np.pi * hour/24)
    sin_hour = np.sin(2 * np.pi * hour/24)

    return cos_hour, sin_hour

cos_hour, sin_hour = getCosSinHour(data)
# data['cos_hour'] = cos_hour
# data['sin_hour'] = sin_hour

#convert daynight to 1 and 0 and drop unwanted rows in that column
#convert every element in daynight to int is possible


#convert every element in daynight to int is possible
data['daynight'] = data['daynight'].apply(lambda x: int(x) if x.isdigit() else x)

data['daynight'] = data['daynight'].replace('D', 1)
data['daynight'] = data['daynight'].replace('N', 0)

In [ ]:
target = data['type']
dropped_data = data.drop(['longitude','latitude','acq_date','acq_time','satellite','instrument','type','confidence','version','precipitation_sum','frp'], axis=1)#,'daynight'
dropped_data

In [ ]:
target.replace({0: 1, 1: 0, 2: 0,3:0},inplace=True)
target

In [ ]:
dropped_data.shape

In [ ]:

# x_train_reshaped = np.expand_dims(dropped_data.values, axis=1)


# n_samples, timesteps, n_features = x_train_reshaped.shape
# x_train_reshaped_2d = x_train_reshaped.reshape(n_samples, timesteps * n_features)

# Apply SMOTE
smote = SMOTE(sampling_strategy='not majority', random_state=42)
x_train_balanced, y_train_balanced = smote.fit_resample(dropped_data, target)

# Reshape back to 3D
#x_train_balanced = x_train_balanced.reshape(-1, timesteps, n_features)

# Verify the new shape
print("Balanced x_train shape:", x_train_balanced.shape)
print("Balanced y_train shape:", y_train_balanced.shape)

x_train, x_test, y_train, y_test = train_test_split(x_train_balanced, y_train_balanced, test_size=0.2, random_state=42,stratify=y_train_balanced)
x_train,x_val,y_train, y_val = train_test_split(x_train,y_train, test_size=0.2, random_state=42,stratify=y_train)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(dropped_data,target, test_size=0.2, random_state=42,stratify=target)
x_train,x_val,y_train, y_val = train_test_split(x_train,y_train, test_size=0.2, random_state=42,stratify=y_train)

In [ ]:
#save the scalers
scalers = {}

for column in x_train.columns:
    scaler = MinMaxScaler()
    x_train[[column]] = scaler.fit_transform(x_train[[column]])
    scalers[column] = scaler

for column, scaler in scalers.items():
    joblib.dump(scaler, f"{column}_scaler.pkl")

In [ ]:
# Load the saved scalers
scalers = {}
for column in x_train.columns:

    if column == 'daynight':
        continue
    scaler = joblib.load(f"{column}_scaler.pkl")
    scalers[column] = scaler

# Transform the validation and test features using the loaded scalers
x_val_scaled = x_val.copy()  
x_train_scaled = x_train.copy()
x_test_scaled = x_test.copy()  
for column, scaler in scalers.items():
    x_train_scaled[[column]] = scaler.transform(x_train[[column]])
    x_val_scaled[[column]] = scaler.transform(x_val[[column]])
    x_test_scaled[[column]] = scaler.transform(x_test[[column]])

In [ ]:
x_train_scaled

# ***Model Training***

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
np.__version__#'1.21.5'

In [ ]:
!pip install -Iv numpy==1.19.5 --user

In [ ]:
### LSTM
def reshape_data_for_lstm(x_train, x_test_scaled, x_val_scaled):
    x_train_array = x_train.values
    x_train_reshaped = np.expand_dims(x_train_array, axis=1)

    x_test_array = x_test_scaled.values
    x_test_reshaped = np.expand_dims(x_test_array, axis=1)

    x_val_array = x_val_scaled.values
    x_val_reshaped = np.expand_dims(x_val_array, axis=1)

    input_shape = (1, x_train_reshaped.shape[2]) #for LSTM
    return x_train_reshaped, x_test_reshaped, x_val_reshaped, input_shape



###cnn
def reshape_data_for_cnn(x_train, x_test_scaled, x_val_scaled):
    x_train_array = x_train.values
    x_train_reshaped = np.expand_dims(x_train_array, axis=-1)

    x_test_array = x_test_scaled.values
    x_test_reshaped = np.expand_dims(x_test_array, axis=-1)

    x_val_array = x_val_scaled.values
    x_val_reshaped = np.expand_dims(x_val_array, axis=-1)

    input_shape = (x_train_reshaped.shape[1],1) #cnn
    return x_train_reshaped, x_test_reshaped, x_val_reshaped, input_shape


x_train_reshaped_lstm, x_test_reshaped_lstm, x_val_reshaped_lstm, input_shape_lstm = reshape_data_for_lstm(x_train_scaled,  x_test_scaled, x_val_scaled)

x_train_reshaped, x_test_reshaped, x_val_reshaped, input_shape = reshape_data_for_cnn(x_train_scaled, x_test_scaled, x_val_scaled)


In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras.regularizers import l2

#lstm
def build_model_lstm(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('input_units', min_value=32, max_value=512, step=32),
                   return_sequences=True,
                   input_shape=input_shape_lstm)) 
    model.add(BatchNormalization())

    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(LSTM(units=hp.Int(f'lstm_{i}_units', min_value=32, max_value=512, step=32), return_sequences=True,kernel_regularizer=l2(hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='log'))))
        model.add(BatchNormalization())

    model.add(LSTM(units=hp.Int('layer_2_neurons', min_value=32, max_value=512, step=32), return_sequences=True,kernel_regularizer=l2(hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='log'))))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0, max_value=0.5, step=0.1)))
    #model.add(Dense(units=y_train.shape[1], activation=hp.Choice('dense_activation', values=['relu', 'sigmoid'], default='relu')))
    model.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
#cnn
def build_model_cnn(hp):
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(filters=hp.Int('input_filters', min_value=32, max_value=512, step=32),
                     kernel_size=1,#hp.Int('input_kernel_size', min_value=1, max_value=7, step=2),
                     activation='relu',
                     input_shape=input_shape,
                     kernel_regularizer=l2(hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='log'))))
    model.add(BatchNormalization())
    
    # Additional Conv1D layers based on n_layers
    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(Conv1D(filters=hp.Int(f'conv_{i}_filters', min_value=32, max_value=512, step=32),
                         kernel_size=1,#hp.Int(f'conv_{i}_kernel_size', min_value=3, max_value=7, step=2),
                         activation='relu',
                         kernel_regularizer=l2(hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='log'))))
        model.add(BatchNormalization())

    # Final Conv1D layer
    model.add(Conv1D(filters=hp.Int('final_conv_filters', min_value=32, max_value=512, step=32),
                     kernel_size=1,#p.Int('final_conv_kernel_size', min_value=3, max_value=7, step=2),
                     activation='relu',
                     kernel_regularizer=l2(hp.Float('l2_reg', min_value=1e-6, max_value=1e-2, sampling='log'))))
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0, max_value=0.5, step=0.1)))
    
    # Flatten the output before passing it to Dense layers
    model.add(Flatten())
    
    # Dense layer for binary classification
    model.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # Monitor the validation loss
    patience=3, # Stop after 5 epochs with no improvement
    mode='min', # Minimize the loss
    restore_best_weights=True # Restore the weights from the epoch with the best validation loss
)


# Set up the Keras Tuner
tuner = RandomSearch(
    build_model_lstm,
    objective='val_loss',
    max_trials=4,
    directory='./tuner',
    project_name='/lstm',
    executions_per_trial=2,
    overwrite=True
)


# Run the hyperparameter search
tuner.search(
    x=x_train_reshaped_lstm,
    y=y_train.values.reshape(-1,1),
    epochs=30,
    batch_size=32,
    validation_data=(x_test_reshaped_lstm, y_test.values.reshape(-1, 1)),
    callbacks=[early_stopping],
)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_cnn1 = load_model('models/cnn_1rst_88acc.h5')
model_cnn2 = load_model('models/cnn_daynight_89acc.h5')
model_lstm1 = load_model('models/lstm_model_89acc.h5')
model_lstm2 = load_model('models/lstm_daynight_90acc.h5')

# Models


In [ ]:
best_models = tuner.get_best_models(num_models=1)[0]
tuner.get_best_models(num_models=1)[0].summary()

In [ ]:
best_models = tuner.get_best_models(num_models=1)[0]
tuner.get_best_models(num_models=1)[0].summary()

In [ ]:
best_models = tuner.get_best_models(num_models=1)[0]
tuner.get_best_models(num_models=1)[0].summary()

In [ ]:
best_models = tuner.get_best_models(num_models=1)[0]  # Specify the number of best models you want to retrieve

# Print the summary of the best model and store the model weigths to use for prediction
# model = 0
# for i in best_models:
#     print(i.summary())
#     model = i

In [ ]:
x_val_reshaped.shape

In [ ]:
inference = best_models.evaluate(x_val_reshaped, y_val.values.reshape(-1,1))

print("loss:", inference[0])
print("accuracy:", inference[1])

In [ ]:
y_pred = best_models.predict_classes(x_val_reshaped)

# Compute the confusion matrix
cm = confusion_matrix( y_val.values.reshape(-1,1), y_pred.reshape(-1,1))# for lstm y_val.values.reshape(-1,1), y_pred.reshape(-1,1) #for cnn y_val, y_pred
 
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', annot_kws={"size": 12})
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import auc, roc_curve


y_pred_prob = best_models.predict(x_test_reshaped).ravel()

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
x_val.shape

In [ ]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

y_pred_test = best_models.predict_classes(x_val_reshaped).reshape(-1,1)

precision = precision_score(y_val, y_pred_test)
recall = recall_score(y_val, y_pred_test)
f1 = f1_score(y_val, y_pred_test)


print("Classification Report:")
print(classification_report(y_val, y_pred_test))

# Print precision, recall, and F1 score
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

## ***Ensemble Prediction***

In [ ]:
import pandas as pd 
# import traintestsplit
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
def predDataForModelWithDaynight(mode):
    data = pd.read_csv(r'./merged_data.csv')

    data['daynight'] = data['daynight'].replace('D', 1)
    data['daynight'] = data['daynight'].replace('N', 0)
    target = data['type']

    if mode == 'daynight':
        dropped_data = data.drop(['longitude','latitude','acq_date','acq_time','satellite','instrument','type','confidence','version','precipitation_sum','frp'], axis=1)#,'daynight'
    else:
        dropped_data = data.drop(['longitude','latitude','acq_date','acq_time','satellite','instrument','type','daynight','confidence','version','precipitation_sum','frp','daynight'], axis=1)

    target.replace({0: 1, 1: 0, 2: 0,3:0},inplace=True)

    x_train, x_test, y_train, y_test = train_test_split(dropped_data,target, test_size=0.2, random_state=42,stratify=target)
    x_train,x_val,y_train, y_val = train_test_split(x_train,y_train, test_size=0.2, random_state=42,stratify=y_train)

    # Load the saved scalers
    scalers = {}
    for column in x_train.columns:

        if column == 'daynight':
            continue
        scaler = joblib.load(f"{column}_scaler.pkl")
        scalers[column] = scaler

    # Transform the validation and test features using the loaded scalers
    x_val_scaled = x_val.copy()
    x_train_scaled = x_train.copy()
    x_test_scaled = x_test.copy()
    for column, scaler in scalers.items():
        x_train_scaled[[column]] = scaler.transform(x_train[[column]])
        x_val_scaled[[column]] = scaler.transform(x_val[[column]])
        x_test_scaled[[column]] = scaler.transform(x_test[[column]])

    x_train_reshaped, x_test_reshaped, x_val_reshaped, input_shape = reshape_data_for_cnn(x_train_scaled, x_test_scaled, x_val_scaled)
    x_train_reshaped_lstm, x_test_reshaped_lstm, x_val_reshaped_lstm, input_shape = reshape_data_for_lstm(x_train_scaled, x_test_scaled, x_val_scaled)

    return x_val_reshaped, x_val_reshaped_lstm, y_val

x_val_reshaped_daynight, x_val_reshaped_lstm_daynight, y_val = predDataForModelWithDaynight(mode='daynight')
x_val_reshaped, x_val_reshaped_lstm, y_val = predDataForModelWithDaynight(mode='')


In [ ]:
x_val_reshaped_daynight.shape

In [ ]:
x_val_reshaped_daynight = np.asarray(x_val_reshaped_daynight).astype('float32')
x_val_reshaped_lstm_daynight = np.asarray(x_val_reshaped_lstm_daynight).astype('float32')

In [ ]:
from tensorflow.keras.models import load_model

model_1 = load_model(r'F:\code\cnn_1rst_88acc.h5')
model_2 = load_model(r'F:\code\cnn_daynight_89acc.h5')
model_3 = load_model(r'F:\code\lstm_model_89acc.h5')
model_4 = load_model(r'F:\code\lstm_daynight_90acc.h5')

#perform majority voting
y_pred_1 = model_1.predict_classes(x_val_reshaped)
y_pred_2 = model_2.predict_classes(x_val_reshaped_daynight)
y_pred_3 = model_3.predict_classes(x_val_reshaped_lstm)
y_pred_4 = model_4.predict_classes(x_val_reshaped_lstm_daynight)

#perform majority voting for the ensemble model while outputs are classes


# #give classification report
# print("Classification Report:")
# print(classification_report(y_val, y_pred_ensemble))

In [ ]:
 
def vote(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

y_ens_pred = []
count = []
for i in range(len(y_pred_1)):
    count.append(y_pred_1[i])
    #count.append(y_pred_2[i])
    count.append(y_pred_3[i])
    count.append(y_pred_4[i])
    y_ens_pred.append(vote(count))
    count = []

In [ ]:
print("Classification Report: model 1")
y_pred_1 = [int(x) for x in y_pred_1]
print(classification_report(y_val, y_pred_1))

print("Classification Report: model 2")
y_pred_2 = [int(x) for x in y_pred_2]
print(classification_report(y_val, y_pred_2))

print("Classification Report: model 3")
y_pred_3 = [int(x) for x in y_pred_3]
print(classification_report(y_val, y_pred_3))

print("Classification Report: model 4")
y_pred_4 = [int(x) for x in y_pred_4]
print(classification_report(y_val, y_pred_4))



In [ ]:
#import classification report
from sklearn.metrics import classification_report

#give classification report
print("Classification Report: 2,3,4")
print(classification_report(y_val, y_ens_pred))

## ***Mixture Of Experts***

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, BatchNormalization, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
import numpy as np
from kerastuner import HyperParameters

In [ ]:
input_shape

In [ ]:
def build_expert_model_cnn(input_shape,num_of_hidden):
    model = Sequential()
    
    # First Conv1D layer
    model.add(Conv1D(filters=64,
                     kernel_size=1,
                     activation='relu',
                     input_shape=input_shape,
                     kernel_regularizer=l2(1e-2)))
    model.add(BatchNormalization())
    
    # Additional Conv1D layers
    for i in range(num_of_hidden):
        model.add(Conv1D(filters=64,
                         kernel_size=1,
                         activation='relu',
                         kernel_regularizer=l2(1e-2)))
        model.add(BatchNormalization())

    # Final Conv1D layer
    model.add(Conv1D(filters=64,
                     kernel_size=1,
                     activation='relu',
                     kernel_regularizer=l2(1e-4)))
    model.add(Dropout(rate=0.3))
    
    # Flatten the output before passing it to Dense layers
    model.add(Flatten())
    
    # Dense layer for binary classification
    model.add(Dense(units=1, activation='softmax'))  # Assuming binary classification
    
    return model

#create similar lstm model
def build_expert_model_lstm(input_shape,num_of_hidden):
    model = Sequential()
    model.add(LSTM(units=64,
                   return_sequences=True,
                   input_shape=input_shape)) 
    model.add(BatchNormalization())

    for i in range(num_of_hidden):
        model.add(LSTM(units=64, return_sequences=True,kernel_regularizer=l2(1e-4)))
        model.add(BatchNormalization())

    model.add(LSTM(units=64, return_sequences=True,kernel_regularizer=l2(1e-4)))
    model.add(Dropout(rate=0.3))
    #add flatten layer 
    model.add(Flatten())
    model.add(Dense(units=1, activation='softmax'))  # Assuming binary classification
    
    return model

# expert1 = build_expert_model_cnn(input_shape,3)
# expert2 = build_expert_model_cnn(input_shape,4)
expert1 = build_expert_model_lstm(input_shape_lstm,5) 
expert3 = build_expert_model_lstm(input_shape_lstm,3)
expert4 = build_expert_model_lstm(input_shape_lstm,4)
experts = [expert1,  expert3, expert4]#expert2,

#to kalytero einai 3 cnn me 3,4,5 num of hidden layers.

In [ ]:
def build_gating_network(input_shape, num_experts):
    inputs = Input(shape=input_shape)
    x = Flatten()(inputs)
    x = Dense(64, activation='relu',kernel_regularizer=l2(1e-2))(x)
    #x = Dropout(0.3)(x)
    x = Dense(32, activation='relu',kernel_regularizer=l2(1e-2))(x)
    #x = Dropout(0.2)(x)
    outputs = Dense(num_experts, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

num_experts = len(experts)+1
gating_network = build_gating_network(input_shape_lstm, num_experts)

In [ ]:
def mixture_of_experts(inputs, experts, gating_network):
    gating_weights = gating_network(inputs)
    
    expert_outputs = [expert(inputs) for expert in experts]
    expert_outputs = tf.stack(expert_outputs, axis=-1)
    
    
    weighted_expert_outputs = tf.reduce_sum(expert_outputs * tf.expand_dims(gating_weights, -1), axis=-1)
    
    return weighted_expert_outputs

inputs = Input(shape=input_shape_lstm)
outputs = mixture_of_experts(inputs, experts, gating_network)
moe_model = Model(inputs, outputs)

In [ ]:

# Compile and train the model
moe_model.compile(loss="sparse_categorical_crossentropy", optimizer='adam',learning_rate=2e-4, metrics=['accuracy'])
moe_model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', # Monitor the validation loss
    patience=3, # Stop after 5 epochs with no improvement
    mode='min', # Minimize the loss
    restore_best_weights=True # Restore the weights from the epoch with the best validation loss
)
#add reducelr on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05,
                              patience=3, min_lr=1e-6)


# Set up the Keras Tuner
# tuner = RandomSearch(
#     build_model_lstm,
#     objective='val_loss',
#     max_trials=4,
#     executions_per_trial=2,
#     overwrite=True
# )


# # Run the hyperparameter search
# tuner.search(
#     x=x_train_reshaped,
#     y=y_train.values.reshape(-1,1),
#     epochs=30,
#     batch_size=32,
#     validation_data=(x_test_reshaped, y_test.values.reshape(-1, 1)),
#     callbacks=[early_stopping],
# )



moe_model.fit(x_train_reshaped_lstm, y_train.values.reshape(-1,1),#
              validation_data=(x_test_reshaped_lstm, y_test.values.reshape(-1, 1)),#
                epochs=30, batch_size=32,callbacks=[early_stopping,reduce_lr],verbose=2)


In [ ]:
moe_model.evaluate(x_val_reshaped, y_val.values.reshape(-1,1))

In [ ]:
y_pred.reshape(-1,1).shape

In [ ]:
predicted_prob = moe_model.predict(x_val_reshaped_lstm)
y_pred_moe = np.argmax(predicted_prob, axis=1)
#y_pred_moe = moe_model.predict(x_val_reshaped_lstm)

# Compute the confusion matrix
cm = confusion_matrix( y_val.values.reshape(-1,1), y_pred_moe.reshape(-1,1))# for lstm y_val.values.reshape(-1,1), y_pred.reshape(-1,1) #for cnn y_val, y_pred
 
# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', annot_kws={"size": 12})
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
#import classification report
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred_moe))